In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../..')

In [5]:
import pandas as pd
import deepmatcher as dm
from utils.explainer import Explainer
from utils.patternDiscovery import mineRules

In [19]:
pos_nn = pd.read_csv('experiment_results/testposNN_RNN.csv')
negatives = pd.read_csv('../../Structured/Amazon-Google/negatives.csv')
association_rules = mineRules(pos_nn,negatives,'title',min_confidence=0.8,min_support=0.1)
association_rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedents_isleft,consequents_isright
3,(L_microsoft),(R_microsoft),0.243590,0.235043,0.235043,0.964912,4.105263,0.177789,21.801282,True,True
18,(L_adobe),(R_adobe),0.128205,0.175214,0.106838,0.833333,4.756098,0.084374,4.948718,True,True
23,(L_office),(R_microsoft),0.111111,0.235043,0.102564,0.923077,3.927273,0.076448,9.944444,True,True
25,"(L_office, L_microsoft)",(R_microsoft),0.106838,0.235043,0.102564,0.960000,4.084364,0.077453,19.123932,True,True


In [10]:
association_rules_onprice = mineRules(pos_nn,negatives,'price',min_confidence=0.5,min_support=0.05)
association_rules_onprice.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedents_isleft,consequents_isright


In [22]:
len(pos_nn)

234

In [33]:
train = pd.read_csv('../../Structured/Amazon-Google/merged_train.csv')
price_distribution = dict(train.groupby(by=['ltable_price','rtable_price']).size())

## Test patterns

In [20]:
rnn_model = dm.MatchingModel(attr_summarizer='rnn')
rnn_model.load_state('../../models/amazongoogle_rnn.pth')
explainer = Explainer(rnn_model,10)

In [21]:
%%capture
explanations = explainer.generateExplanations(pos_nn,negatives,'price',0.03,
                                              '../../Structured/Amazon-Google/test_positives.csv',1)

In [22]:
for df,lval,rval in explanations:
    print((lval,rval))
    print(len(df))

('495.0', '49.99')
141
('29.99', '61.8')
104
('99.99', '85.07')
109
('nan', '21.41')
151
('499.99', '219.99')
93
('89.99', '621.99')
171


In [ ]:
p1 =['microsoft windows','microsoft windows']
explainer.testRobustness(testset_path='../../Structured/Amazon-Google/test_positives.csv',
                         true_label=1,attribute='title',substitute_values=p1,
                         strategy='concat')

In [12]:
import numpy as np
p2 = ['239.99','109.99']
explainer.testRobustness(testset_path='../../Structured/Amazon-Google/positives.csv',
                         true_label=1,attribute='price',substitute_values=p2,
                         strategy='replace')

===>  PREDICT Epoch 18
Finished Epoch 18 || Run Time:    0.6 | Load Time:    1.0 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s:   0.00

===>  PREDICT Epoch 18
Finished Epoch 18 || Run Time:    0.6 | Load Time:    1.1 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s:   0.00



(990, 1009)

## Inspect NN on critical attributes for negatives

In [7]:
DATASET_DIR = '../../datasets/Structured/Amazon-Google/'
pos_nn_hybrid = pd.read_csv('experiment_results/testpos_nn_hybrid.csv')
negatives = pd.read_csv(DATASET_DIR+'negatives.csv')

In [23]:
association_rules = mineRules(pos_nn_hybrid,negatives,'title',min_confidence=0.8,min_support=0.1)
association_rules = association_rules.sort_values(by=['support'],ascending=False)
association_rules.head(20)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedents_isleft,consequents_isright
1316,(L_adobe),(R_adobe),0.162393,0.153846,0.153846,0.947368,6.157895,0.128863,16.076923,True,True
1320,(L_adobe),(R_cs3),0.162393,0.145299,0.145299,0.894737,6.157895,0.121704,8.119658,True,True
1341,(L_cs3),(R_cs3),0.145299,0.145299,0.145299,1.000000,6.882353,0.124187,inf,True,True
1347,"(L_adobe, L_cs3)",(R_cs3),0.145299,0.145299,0.145299,1.000000,6.882353,0.124187,inf,True,True
1327,(L_adobe),"(R_cs3, R_adobe)",0.162393,0.136752,0.136752,0.842105,6.157895,0.114545,5.467236,True,True
1344,(L_cs3),(R_adobe),0.145299,0.153846,0.136752,0.941176,6.117647,0.114398,14.384615,True,True
1352,"(L_adobe, L_cs3)",(R_adobe),0.145299,0.153846,0.136752,0.941176,6.117647,0.114398,14.384615,True,True
1362,(L_cs3),"(R_cs3, R_adobe)",0.145299,0.136752,0.136752,0.941176,6.882353,0.116882,14.675214,True,True
1369,"(L_adobe, L_cs3)","(R_cs3, R_adobe)",0.145299,0.136752,0.136752,0.941176,6.882353,0.116882,14.675214,True,True
340,"(L_adobe, L_], L_[)",(R_cs3),0.119658,0.145299,0.115385,0.964286,6.636555,0.097998,23.931624,True,True


In [20]:
hybrid_model = dm.MatchingModel(attr_summarizer='hybrid')
hybrid_model.load_state('../../models/amazongoogle_hybrid.pth')

In [21]:
hybridExp = Explainer(hybrid_model,10)

In [22]:
hybridExp.testRobustness('../../datasets/Structured/Amazon-Google/test_positives.csv',1,'title',
                        ['adobe mac cs3','adobe'],strategy='concat')


Reading and processing data from "../../datasets/Structured/Amazon-Google/test_positives.csv"
0% [############################# ] 100% | ETA: 00:00:00

===>  PREDICT Epoch 2


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00

Reading and processing data from "temp/new_test.csv"


Finished Epoch 2 || Run Time:    0.5 | Load Time:    0.3 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s:   0.00



0% [############################# ] 100% | ETA: 00:00:00

===>  PREDICT Epoch 2


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 2 || Run Time:    0.5 | Load Time:    0.3 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s:   0.00



(218, 215)